In [1]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
import random
import os

Using TensorFlow backend.


In [2]:
FAST_RUN = False
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=[IMAGE_WIDTH, IMAGE_HEIGHT]
IMAGE_CHANNELS=3

# Reading Image from Dir and creating .csv for lables 

In [3]:
filenames = os.listdir("train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [4]:
df

,filename,category
0,dog.4490.jpg,1
1,cat.3202.jpg,0
2,cat.3288.jpg,0
3,cat.581.jpg,0
4,dog.2096.jpg,1
...,...,...
24995,dog.1974.jpg,1
24996,dog.9740.jpg,1
24997,cat.3277.jpg,0
24998,dog.986.jpg,1


# Downloading MobileNet model for Transfer learning

In [5]:
from keras.applications.mobilenet import MobileNet


MobileNet =  MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
for layer in MobileNet.layers:
    layer.trainable = False

In [7]:
x = Flatten()(MobileNet.output)

In [8]:
prediction = Dense(2, activation='softmax')(x)

In [9]:
model = Model(inputs=MobileNet.input, outputs=prediction)

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

# Setting Callback Functions 

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [12]:
earlystop = EarlyStopping(patience=10)

In [13]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [14]:
callbacks = [earlystop, learning_rate_reduction]

# Replacing lables  o To cat and 1 to Dog 

In [15]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 

In [16]:
df

,filename,category
0,dog.4490.jpg,dog
1,cat.3202.jpg,cat
2,cat.3288.jpg,cat
3,cat.581.jpg,cat
4,dog.2096.jpg,dog
...,...,...
24995,dog.1974.jpg,dog
24996,dog.9740.jpg,dog
24997,cat.3277.jpg,cat
24998,dog.986.jpg,dog


# Spliting df to create training_df and validation_df 

In [17]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [18]:
train_df

,filename,category
0,cat.6687.jpg,cat
1,cat.8593.jpg,cat
2,cat.1302.jpg,cat
3,dog.8711.jpg,dog
4,dog.2839.jpg,dog
...,...,...
19995,dog.2777.jpg,dog
19996,dog.8366.jpg,dog
19997,dog.3801.jpg,dog
19998,dog.7174.jpg,dog


In [19]:
total_train = train_df.shape[0] # 20000
total_validate = validate_df.shape[0] # 500
batch_size=15

# Loading traning and Validation image data using flow_from_dataframe (df)

In [20]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "train/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 20000 validated image filenames belonging to 2 classes.


In [21]:
train_generator.class_indices.items()

dict_items([('cat', 0), ('dog', 1)])

In [22]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "train/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 5000 validated image filenames belonging to 2 classes.


In [23]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Tranning the model 

In [24]:
epochs=2


history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)



Epoch 1/2
1333/1333 [==============================] - 642s 481ms/step - loss: 1.6074 - accuracy: 0.9363 - val_loss: 0.0000e+00 - val_accuracy: 0.9644
Epoch 2/2


/home/nitin/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1333/1333 [==============================] - 632s 474ms/step - loss: 1.8699 - accuracy: 0.9515 - val_loss: 0.0000e+00 - val_accuracy: 0.9819


In [25]:
Accuracy = history.history['accuracy']
Accuracy

Accuracy =  Accuracy.pop()
Accuracy

Accuracy = Accuracy*100
Accuracy


[0.93630224, 0.9514636]

0.9514636

95.14635801315308

95.0

In [29]:
Accuracy = str(Accuracy)
Accuracy

'95.0'

In [30]:
Accuracy = "Model_Accuracy_" + Accuracy + "%" 

In [31]:
Accuracy

'Model_Accuracy_95.0%'

In [32]:
def save_model_to_db(model, client, db, dbconnection, model_name):
    import pickle
    import time
    import pymongo
    #pickling the model
    pickled_model = pickle.dumps(model)
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    info = mycon.insert_one({model_name: pickled_model, 'name': model_name, 'created_time':time.time()})
    print(info.inserted_id, ' saved with this id successfully!')
    
    details = {
        'inserted_id':info.inserted_id,
        'model_name':model_name,
        'created_time':time.time()
    }
    print(details)
    return details

In [33]:
details = save_model_to_db(model = model, client ='mongodb://localhost:27017/', db = 'Cat_dog_classifier', 
                 dbconnection = Accuracy, model_name = "CNN")

5eb7e0806b499e4ad27c23c3  saved with this id successfully!
{'inserted_id': ObjectId('5eb7e0806b499e4ad27c23c3'), 'model_name': 'CNN', 'created_time': 1589108864.459663}
